In [1]:
from vedo import Points, Plotter, Line, Axes, Plane, Grid
from IPython.display import Video
import imageio.v2 as imageio
import os
import shutil
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
def generate_video_from_raw_data(file_name, raw_data_folder, prints = False):

    if not prints:
        print("Generating video...")
        print("May take a while: use prints = True to see progress...")
    
    # Load data
    marker_data = pd.read_table(raw_data_folder+file_name+".tsv", 
                        sep = '\t',
                        skiprows = range(0, 11)
                        )
    
    marker_data = marker_data.drop(marker_data.columns[marker_data.columns.str.contains('unnamed', case=False)], axis=1)
    
    marker_start_time = pd.read_table(raw_data_folder+file_name+".tsv",
                                      skiprows = [*range(0, 7), 8], 
                                      header=None,
                                      on_bad_lines = 'skip').to_numpy()

    print(marker_start_time)
    
    marker_start_time = datetime.strptime(marker_start_time[:, 1][0], '%Y-%m-%d, %H:%M:%S.%f').timestamp() # add an hour for daylight savings and some adjustment
    
    marker_data_np = marker_data.drop(["Time", "Frame"], axis=1).to_numpy()
    
    reshaped = marker_data_np.reshape(marker_data_np.shape[0], -1, 3)
    reshaped[:, :, [1, 2]] = reshaped[:, :, [2, 1]]
    marker_data_np = reshaped.reshape(marker_data_np.shape)

    if prints:
        print(f"Raw data shape: {marker_data_np.shape}")
        print()
    
    # Calculate number of points per frame (each point has 3 coords: x,y,z)
    num_points = marker_data_np.shape[1] // 3
    
    # If folder exists, delete it completely
    if os.path.exists("output"):
        shutil.rmtree("output")
    
    # Create empty folder
    os.makedirs("output")
    
    # Initialize Vedo Plotter
    vp = Plotter(offscreen=True, bg='white', size=(800, 608))
    
    # Set camera view
    vp.camera.SetPosition(8, 8, 8)
    vp.camera.SetFocalPoint(0, 0, 0)
    vp.camera.SetViewUp(0, 1, 0)
    vp.render()
    
    grid = Grid(pos=(0,0,0), s=(100,100), res=(200,200), c='lightgray', alpha=0.3)
    grid.rotate(angle=90, axis=(1,0,0))  # rotate 90 degrees around X axis
    vp.add(grid)
    
    # format column names for selection
    point_names = marker_data.columns.to_list()
    point_names = [x for x in point_names if x[-1] not in ["Y", "Z"]]
    point_names.remove("Time")
    point_names.remove("Frame")
    point_names = [x[:-2] for x in point_names]

    connections = [
        # hip
        (point_names.index('BACK_REF'), point_names.index('BACK_X')),
        (point_names.index('BACK_X'), point_names.index('BACK_O')),
        (point_names.index('BACK_O'), point_names.index('BACK_Y')),
        #right foot
        (point_names.index('R_TOE'), point_names.index('R_MET5')),
        (point_names.index('R_TOE'), point_names.index('R_HEEL')),
        (point_names.index('R_TOE'), point_names.index('R_REF')),
        (point_names.index('R_REF'), point_names.index('R_MET5')),
        (point_names.index('R_REF'), point_names.index('R_HEEL')),
        (point_names.index('R_MET5'), point_names.index('R_HEEL')),
        #left foot
        (point_names.index('L_TOE'), point_names.index('L_MET5')),
        (point_names.index('L_TOE'), point_names.index('L_HEEL')),
        (point_names.index('L_TOE'), point_names.index('L_REF')),
        (point_names.index('L_REF'), point_names.index('L_MET5')),
        (point_names.index('L_REF'), point_names.index('L_HEEL')),
        (point_names.index('L_MET5'), point_names.index('L_HEEL')),
        #wrist    
        (point_names.index('WRIST'), point_names.index('WRIST_LAT')),
        (point_names.index('WRIST_MED'), point_names.index('WRIST_LAT')),
    ]

    # list to hold things to be deleted
    dynamic_actors = []
    
    # Animation loop
    for i, frame in enumerate(marker_data_np):
        pts_coords = frame.reshape((num_points, 3)) / 1000 # convert from mm to m

        if prints:
            print(f"Frame {i}, points shape: {pts_coords.shape}, first point: {pts_coords[0]}")
    
        # Remove previous dynamic actors
        for actor in dynamic_actors:
            vp.remove(actor)
        dynamic_actors.clear()
    
        # Create points and lines
        pts = Points(pts_coords, r=3, c='blue')
    
        for idx1, idx2 in connections:
            p1 = pts_coords[idx1]
            p2 = pts_coords[idx2]
        
            if not(np.allclose(p1, 0)) and not(np.allclose(p2, 0)):
                line = Line(p1, p2, c='black', lw=2)
                vp.add(line)
                dynamic_actors.append(line)
            else:
                # Optionally: log skipped lines
                if prints:
                    print(f"Skipping line between {idx1} and {idx2} due to missing data")
    
        # Add to scene
        vp.add(pts, line)
        dynamic_actors.extend([pts, line])
    
        # Render and save frame
        vp.render()
        vp.screenshot(f"output/frame_{i:04d}.png")
    
    # Finalize
    vp.close()

    # Create video from saved frames
    image_folder = "output"
    fps = 100  # frames per second
    
    images = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")])
    with imageio.get_writer(raw_data_folder+file_name+".mp4", fps=fps) as writer:
        for filename in images:
            image_path = os.path.join(image_folder, filename)
            image = imageio.imread(image_path)
            writer.append_data(image)

    print(f"Video saved to {raw_data_folder+file_name+".mp4"}")
    print()
    
    # get rid of outputs folder
    if os.path.exists("output"):
        shutil.rmtree("output")

In [3]:
raw_data_folder = r"C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\718\Lab\Motion Capture Data\\"

tsv_files = [
    os.path.splitext(f)[0]
    for f in os.listdir(raw_data_folder)
    if f.endswith('.tsv') and not f.startswith('synced')
]

for file_name in tsv_files:
    generate_video_from_raw_data(file_name, raw_data_folder, prints = False)
    Video(raw_data_folder+file_name+".mp4", embed=True, width=608)

Generating video...
May take a while: use prints = True to see progress...
[['TIME_STAMP' '2025-09-18, 11:00:12.355' 7506.2173958]]
Video saved to C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\718\Lab\Motion Capture Data\\10m_walk.mp4

Generating video...
May take a while: use prints = True to see progress...
[['TIME_STAMP' '2025-09-18, 11:01:15.057' 7568.9201031]]
Video saved to C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\718\Lab\Motion Capture Data\\10m_walk_bag.mp4

Generating video...
May take a while: use prints = True to see progress...
[['TIME_STAMP' '2025-09-18, 11:07:03.126' 7916.9881735]]
Video saved to C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\718\Lab\Motion Capture Data\\10m_walk_step.mp4

Generating video...
May take a while: use prints = True to see progress...
[['TIME_STAMP' '2025-09-18, 11:04:52.630' 7786.4928931]]
Video saved to C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\718\Lab\Motion Capture Data\\10m_walk_